In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import random
import json
import gc
from pathlib import Path
from functools import partial
import h5py

import numpy as np
import pandas as pd
import torch
import torchio as tio
import h5py
from ipywidgets import interact
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm
from einops import rearrange
from scipy import ndimage
from PIL import Image


dir2 = os.path.abspath('..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
    
from pipeline.utils import index_unsorted

C:\Users\Cefir\anaconda3\envs\Neurophysiological-Data-Decoding\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Cefir\anaconda3\envs\Neurophysiological-Data-Decoding\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\Cefir\anaconda3\envs\Neurophysiological-Data-Decoding\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\Cefir\anaconda3\envs\Neurophysiological-Data-Decoding\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Cefir\anaconda3\envs\Neurophysiological-Data-Decoding\lib\site-packages\numpy\.libs\libopenblas.xwydx2ikjw2nmtwsfyngfuwkqu3lytcz.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# comparison of embedding only decoding and resblock cascade

dataset_path = Path('D:\\Datasets\\NSD\\')

path1 = Path('D:\\Datasets\\NSD\\derivatives\\results\\ViT-B=32\\embedding\\standard\\subj01\\images\\batch-0')
path2 = Path('D:\\Datasets\\NSD\\derivatives\\results\\ViT-B=32\\group-1\\subj01\\aee0d0f6\\images')
out_path = Path('D:\\Datasets\\NSD\\derivatives\\results\\ViT-B=32\\group-1\\subj01\\aee0d0f6\\derived2')
out_path.mkdir(exist_ok=True)

def get_stim_id(p, pos):
    return int(p.name.split('_')[pos].split('-')[-1])

image_paths1 = [p for p in path1.iterdir() if p.name != 'desktop.ini']
image_paths1.sort(key=partial(get_stim_id, pos=3))
image_paths2 = [p for p in path2.iterdir() if p.name != 'desktop.ini']
image_paths2.sort(key=partial(get_stim_id, pos=0))

stimuli_path = dataset_path / 'nsddata_stimuli' / 'stimuli' / 'nsd' / 'nsd_stimuli.hdf5'
stimulus_images = h5py.File(stimuli_path, 'r')['imgBrick']

stimulus_ids = [get_stim_id(p, pos=1) for p in image_paths2]
X = index_unsorted(stimulus_images, stimulus_ids)

In [3]:
X.shape

(442, 425, 425, 3)

In [4]:
all_recon = []
for i in range(X.shape[0] // 3):
    x_real = X[i * 3]
    x_recon1 = np.stack([np.array(Image.open(image_paths1[i * 3 + j])) for j in range(3)])
    x_recon2 = np.stack([np.array(Image.open(image_paths2[i * 3 + j])) for j in range(3)])
    x_real = np.array(Image.fromarray(x_real).resize((224, 224)))[None]
    all_recon.append(np.concatenate([x_real, x_recon2]))
    #all_recon.append(np.concatenate([np.zeros_like(x_real), x_recon2]))
all_recon = np.stack(all_recon)
print(all_recon.shape)

(147, 4, 224, 224, 3)


In [5]:
num_pad = 5
pad_images = [np.zeros_like(all_recon[0])[None] for i in range(num_pad)]
all_recon = np.concatenate([all_recon, *pad_images])

In [6]:

#out_images = rearrange(all_recon, 'n r h w c -> n h (w r) c'
for i, out_img in enumerate(rearrange(all_recon, '(n1 n2) r h w c -> n1 (r h) (n2 w) c', n2=8)):
    Image.fromarray(out_img).save(out_path / f'recon{i}.png')

In [ ]:
from PIL import Image

recon_path = Path('X:\\drive_data\\images\\single_resblock')
cascade_path = Path('X:\\drive_data\\optimization_results\\vqgan_imagenet_f16_1024\\cascade\\29000da3\\images')

image_ids = ['1443537.022563', '1943899.024131', '2437971.005013', '2882301.014188']

all_images = []
for stim_id in image_ids:
    image_paths = [p for p in recon_path.iterdir() if stim_id in p.name]
    image_paths.sort(key=lambda p: int(p.name.split('_')[0].split('-')[1]))
    images = np.stack([np.array(Image.open(p)) for p in image_paths])
    all_images.append(images)
all_images = np.stack(all_images)

cascades = []
for stim_id in image_ids:
    cascade_img = np.array(Image.open([p for p in cascade_path.iterdir() if stim_id in p.name][0]))
    cascades.append(cascade_img)
cascades = np.stack(cascades)

In [ ]:


plt.figure(figsize=(12, 12))
out_img = rearrange(all_images[:, [0, 3, 6, 9, 11]], 'n r h w c -> (n h) (r w) c')
plt.imshow(out_img)
plt.show()
out_path = Path('G:\\Github Repositories\\Google Drive\\School\\2021 Fall\\CMPUT 652\\Project\\images')
Image.fromarray(out_img).save(out_path / 'resblock_comparison.png')

In [ ]:
out_img = rearrange(cascades, 'n h w c -> (n h) w c')
plt.imshow(out_img)
plt.show()
out_path = Path('G:\\Github Repositories\\Google Drive\\School\\2021 Fall\\CMPUT 652\\Project\\images')
Image.fromarray(out_img).save(out_path / 'cascade_comparison.png')

In [ ]:
dataset_path = Path('D:\\Datasets\\NSD\\')
derivatives_path = dataset_path / 'derivatives'
betas_path = dataset_path / 'nsddata_betas' / 'ppdata'
ppdata_path = dataset_path / 'nsddata' / 'ppdata'
stimulu_path = dataset_path / 'nsddata_stimuli' / 'stimuli' / 'nsd' / 'nsd_stimuli.hdf5'
stimulus_images = h5py.File(stimulu_path, 'r')['imgBrick']
subjects = {f'subj0{i}': {} for i in range(1, 9)}

for subject_name, subject_data in subjects.items():
    responses_file_path = ppdata_path / subject_name / 'behav' / 'responses.tsv'
    subject_data['responses'] = pd.read_csv(responses_file_path, sep='\t',)
    
    # The last 3 sessions are currently held-out for the algonauts challenge
    # remove them for now.
    session_ids = subject_data['responses']['SESSION']
    held_out_mask = session_ids > (np.max(session_ids) - 3)
    subject_data['responses'] = subject_data['responses'][~held_out_mask]

subject_name = 'subj01'

split_name = 'split-01'
split = h5py.File(derivatives_path / 'data_splits' / f'{split_name}.hdf5')
subject_split = split[subject_name]

test_response_mask = subject_split['test_response_mask'][:].astype(bool)
validation_response_mask = subject_split['validation_response_mask'][:].astype(bool)
training_response_mask = ~(test_response_mask | validation_response_mask)

responses = subjects[subject_name]['responses']
response_stimulus_ids = responses['73KID'].to_numpy()


def select_fold_images(stimulus_images, response_mask, response_stimulus_ids):
    response_ids = np.where(response_mask)[0]
    stimulus_ids = response_stimulus_ids[response_ids] - 1
    
    argsort_ids = np.argsort(stimulus_ids)
    unique_results = np.unique(stimulus_ids, return_counts=True)
    unique_stimulus_ids, unique_stimulus_counts = unique_results
    
    X_averaged = stimulus_images[unique_stimulus_ids]
    X = np.repeat(X_averaged, unique_stimulus_counts, axis=0)
    
    return X, X_averaged

X_val, X_val_avg = select_fold_images(stimulus_images, validation_response_mask, response_stimulus_ids)

In [ ]:
clip_results_path = Path('D:\\Datasets\\NSD\\derivatives\\results\\ViT-B=32\\embedding\\standard\\subj01\\images')
in_path = clip_results_path / 'batch-0'
out_path = clip_results_path / 'derived'
def get_stim_id(p):
    return int(p.name.split('_')[3].split('-')[-1])

image_paths = list(in_path.iterdir())
image_paths.sort(key=get_stim_id)

all_recon = []
for i in range(200):
    x_real = X_val[i * 3]
    x_recon = np.stack([np.array(Image.open(image_paths[i * 3 + j])) for j in range(3)])
    x_real = np.array(Image.fromarray(x_real).resize((224, 224)))
    x = np.concatenate([x_real[None], x_recon])
    all_recon.append(x)
all_recon = np.stack(all_recon)

In [ ]:
plt.imshow(all_recon[0, 1])

In [ ]:
all_recon.shape
#out_images = rearrange(all_recon, 'n r h w c -> n h (w r) c'
for i, out_img in enumerate(rearrange(all_recon, '(n1 n2) r h w c -> n1 (r h) (n2 w) c', n2=8)):
    Image.fromarray(out_img).save(out_path / f'recon{i}.png')